# [E-04] 작사가 인공지능 만들기

자연어 처리에 대해 배우기 위해 자동으로 문장을 생성해 주는 인공지능을 만들어 본다. 
작사가 인공지능 만들기라 제목을 지은 이유는 학습에 사용될 텍스트 데이터가 팝송의 가사들이기 때문이다. 
이제까지 학습했던 인공지능과는 다른 방식의 데이터 전처리 과정을 거치는데 특히 Tokenizer 과정이 자연어 데이터 전처리 과정의 핵심인 것 같다. 
자연어 처리는 처음 해보기 때문에 많이 헤맬것이라 예상된다. 

오늘의 학습은 아래와 같은 순서로 진행한다. 

    1. 데이터 전처리
    2. 모델 설계
    3. 파라미터 조정 및 모델 학습
    4. 모델 성능 평가 

시작해보자. 

## 1-1. 데이터 전처리

### 필요한 Library import

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob

### 데이터 불러오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = [] # 불러온 데이터를 리스트에 저장

for txt_file in txt_list: 
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:1])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord"]


### 데이터 다듬기

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 첫 인덱스 10까지의 문장 10개를 간단히 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


### 문장 Tokenize 하기 위한 데이터 정리 - 특수기호, 공백, 숫자, 대문자 제거하기

In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 문장이 어떻게 필터링 되는지 확인
print(preprocess_sentence("Hi,  )@(#*$)( my )#(*$)(                         name 23999999 is 1       0982345 Jae."))

<start> hi , my name is jae . <end>


### 정제 데이터 구축

In [5]:
corpus = [] # 정제된 문장을 모을 리스트 생성

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) < 15: continue
    if sentence[-1] == ":": continue # 가수의 이름을 제거합니다.
    if len(set(sentence.split())) < 5: continue # 반복적 요소를 제거하고 길이가 너무 짧은 문장은 제거합니다.
    if len(sentence.split()) > 13: continue # 길이가 너무 긴 문장은 제거합니다.

    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 확인합니다.
corpus[:1000]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>',
 '<start> but if i did , well really , what s it to you ? <end>',
 '<start> there s a blaze of light <end>',
 '<start> it doesn t matter which you heard <end>',
 '<start> the holy or the broken hallelujah hallelujah <end>',
 '<start> hallelujah i did my best , it wasn t mu

후렴구처럼 중복되거나 길이가 너무 짧아 문법을 학습하는데 의미가 없는 단문장들은 모두 제거했다. 최대한 Loss를 줄이려면 데이터 전처리 과정이 가장 중요할 것 같아서 공을 들이고 싶었는데 더 좋은 방법을 찾고 싶었으나 지금은 여기까지가 내 최선이었다. 

### Tokenize 함수 정의

In [6]:

def tokenize(corpus):
    # 원하는 만큼 단어를 기억할 수 있는 tokenizer 제작
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 변환
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    #1 corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    #2 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    #3 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    
    tokenizer.fit_on_texts(corpus) #1
    tensor = tokenizer.texts_to_sequences(corpus) #2
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') #3
    
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   49    4 ...    0    0    0]
 [   2   16 2929 ...    0    0    0]
 [   2   32    7 ...    0    0    0]
 ...
 [   2  245  186 ...    0    0    0]
 [   2  125    4 ...    0    0    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f689c2161f0>


### 생성된 tensor data 출력

In [7]:
print(tensor[:3, :10])

[[   2   49    4   91  281   64   52   10  977 5787]
 [   2   16 2929  881    5    8   11 5326    6  359]
 [   2   32    7   36   15  156  295   28  346    5]]


### tokenizer에 구축된 단어 사전의 인덱스 확인

In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : to
10 : a


### 생성된 tensor를 source와 target으로 분리

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[1])
print(tgt_input[1])

[   2   16 2929  881    5    8   11 5326    6  359    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[  16 2929  881    5    8   11 5326    6  359    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


### data set 객체 생성

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

### 데이터 셋 분리
RNN 학습에서는 test 데이터로 결과를 평가하지 않기 때문에 train, validation set으로만 나누어 준다. 

In [11]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size=0.2,
                                                         shuffle=True,
                                                         random_state=34)

In [12]:
# 데이터 분리 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Val:", enc_val.shape)
print("Target Val:", dec_val.shape)

Source Train: (100215, 32)
Target Train: (100215, 32)
Source Val: (25054, 32)
Target Val: (25054, 32)


노드에 나온 12만개라는 Train set 숫자는 어떻게 구한건지 모르겠다. 주위에 정확히 그 숫자대로 달성한 사람이 없었다. 

## 2. 모델 설계

### 모델 구축

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
              
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


### 데이터를 조금 태워 Model.build() 호출시키기

In [14]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-1.12835623e-04, -5.32116574e-05, -5.51753546e-05, ...,
         -1.12937232e-06, -1.15260620e-04, -1.03486957e-06],
        [-1.09940593e-04, -6.68768280e-06, -1.38564938e-04, ...,
          5.45299517e-05, -1.35684211e-04, -1.76205634e-04],
        [-7.39807947e-05, -2.15651395e-04, -1.03485305e-04, ...,
          1.23575359e-04, -3.40068334e-04, -7.99065092e-05],
        ...,
        [ 1.23443408e-03,  2.98251468e-03,  1.89695472e-03, ...,
          2.05457560e-03,  1.23575400e-03, -3.75968916e-03],
        [ 1.23313337e-03,  2.98766326e-03,  1.85923418e-03, ...,
          2.10044906e-03,  1.26567949e-03, -3.80482269e-03],
        [ 1.23133196e-03,  2.99089192e-03,  1.82688178e-03, ...,
          2.14191154e-03,  1.29232998e-03, -3.84159503e-03]],

       [[-1.12835623e-04, -5.32116574e-05, -5.51753546e-05, ...,
         -1.12937232e-06, -1.15260620e-04, -1.03486957e-06],
        [-1.44446225e-04, -1.19909193e-04, -1

### 모델의 summary 확인하기

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


## 3. 모델 학습

### 학습하기

In [16]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none' )

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/10
392/392 [==============================] - 162s 407ms/step - loss: 1.9374 - val_loss: 1.6525
Epoch 2/10
392/392 [==============================] - 162s 414ms/step - loss: 1.5914 - val_loss: 1.5480
Epoch 3/10
392/392 [==============================] - 163s 415ms/step - loss: 1.5034 - val_loss: 1.4874
Epoch 4/10
392/392 [==============================] - 163s 416ms/step - loss: 1.4403 - val_loss: 1.4425
Epoch 5/10
392/392 [==============================] - 163s 416ms/step - loss: 1.3872 - val_loss: 1.4100
Epoch 6/10
392/392 [==============================] - 163s 416ms/step - loss: 1.3384 - val_loss: 1.3828
Epoch 7/10
392/392 [==============================] - 164s 418ms/step - loss: 1.2932 - val_loss: 1.3554
Epoch 8/10
392/392 [==============================] - 163s 417ms/step - loss: 1.2498 - val_loss: 1.3347
Epoch 9/10
392/392 [==============================] - 163s 417ms/step - loss: 1.2088 - val_loss: 1.3158
Epoch 10/10
392/392 [==============================] - 163s 417m

## 4. 모델 평가

### 자동 문장 완성 함수 작성

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

### 문장 생성해보기

In [23]:
generate_text(model, tokenizer, init_sentence="<start> sorry")

'<start> sorry i m a voodoo chile <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> do not")

'<start> do not let me do <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> why")

'<start> why you wanna be with me <end> '

In [27]:
generate_text(model, tokenizer, init_sentence="<start> dear")

'<start> dear prudence , let me do it <end> '

In [28]:
generate_text(model, tokenizer, init_sentence="<start> why")

'<start> why you wanna be with me <end> '

In [29]:
generate_text(model, tokenizer, init_sentence="<start> but")

'<start> but i m not gonna be <end> '

# 5. 후기

문장을 생성해 보았는데 데이터를 정제하는 과정에서 욕을 필터링 하지 않아 욕이 너무 많이 나왔다. 
예를 들어 'sorry i m motherfXXXing bitXX' 등등 안만드니만 못한 가사를 써내렸다. 
게다가 문장의 완성도도 그리 높지 않아 오래 걸린 시간에 비해 조금 실망했다. 
물론 데이터를 제대로 정제하지 못한 나 스스로에게 아쉬움이 제일 크다.
전체적으로 2가지 부분에서 어려움을 겪었다. 

1. 데이터 전처리 과정에서 텍스트를 깔끔하게 정제하는 과정
2. val_loss가 출력이 되지 않았던 점

첫번째로 텍스트를 깔끔하게 정제하는 과정에서 코딩을 하는 역량이 부족한 것이 정말 아쉬웠다. 어떻게 하고싶은지 머릿속으로 그려졌지만 실제 코드로 구현하지 못했다. 검색과 시행착오를 통해 극복하고자 했으나 Error가 수없이 떴고 시간이 부족해 극복하지 못했다. 두번째로 val_loss가 뜨지 않아 조금 헤맸으나 조원들의 도움으로 해결하였다. 

한번 학습을 돌리는데에도 시간이 유독 오래 걸렸던 프로젝트라 실험을 통해 성능을 개선할 기회가 많이 부족했다. 이럴수록 도메인과 모델의 원리에 대한 이해를 충실히 갖추어야 시간의 제약을 극복하고 더욱 좋은 성능 개선을 이끌어 낼 수 있겠다는 생각이 들었다.
